In [1]:
import os
import openai
import json
from dotenv import load_dotenv

load_dotenv()

True

### Set the OpenAI API Key

In [2]:
openai.api_key = os.environ['OPENAI_API_KEY']

### Define a Get Completion Function

In [22]:
def get_completion(messages, model="gpt-3.5-turbo-1106", temperature=0, max_tokens=300, tools=None):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools
    )
    return response.choices[0].message

### Define Dummy Function

In [33]:
# Defines a dummy function to get the current weather
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather = {
        "location": location,
        "temperature": "50",
        "unit": unit,
    }

    return json.dumps(weather)

### Define Functions

As demonstrated in the OpenAI documentation, here is a simple example of how to define the functions that are going to be part of the request. 

The descriptions are important because these are passed directly to the LLM and the LLM will use the description to determine whether to use the functions or how to use/call.

In [21]:
# define a function as tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },   
    }
]

In [7]:
# define a list of messages

messages = [
    {
        "role": "user",
        "content": "What is the weather like in London?"
    }
]

In [24]:
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zDFJY7MgSkWAvu8h2cFTvdVm', function=Function(arguments='{"location":"London","unit":"celsius"}', name='get_current_weather'), type='function')])


In [27]:
response.tool_calls[0].function.arguments

'{"location":"London","unit":"celsius"}'

We can now capture the arguments:

In [30]:
args = json.loads(response.tool_calls[0].function.arguments)

In [35]:
get_current_weather(**args)

'{"location": "London", "temperature": "50", "unit": "celsius"}'